In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
match_info = ["Season",
              "League",
              "Date",
              "HomeTeam",
              "AwayTeam"]
match_statistics = ["FTHG",
                    "FTAG",
                    "FTR",
                    "HTHG",
                    "HTAG",
                    "HTR",
                    "Attendance",
                    "Referee",
                    "HS",
                    "AS",
                    "HST",
                    "AST",
                    "HHW",
                    "AHW",
                    "HC",
                    "AC",
                    "HF",
                    "AF",
                    "HO",
                    "AO",
                    "HY",
                    "AY",
                    "HR",
                    "AR",
                    "HBP",
                    "ABP"]

In [3]:
def get_measures():
    if os.path.isfile("../data/processed/measures.csv"):
        measures = pd.read_csv("../data/processed/measures.csv")
        measures.Date = pd.to_datetime(measures.Date)
        return measures
    else:
        measures = pd.read_csv("../data/processed/historical_scores.csv", index_col=0)
        measures = measures[match_info]
        measures.Date = pd.to_datetime(measures.Date)
        measures = measures.reset_index()
        measures.to_csv("../data/processed/measures.csv")
        return measures

measures = get_measures()
raw_data = pd.read_csv("../data/processed/historical_scores.csv", index_col=0)
raw_data = raw_data[match_info + match_statistics]
raw_data.Date = pd.to_datetime(raw_data.Date)
raw_data = raw_data.reset_index()

In [10]:
idx = 35000
df = raw_data
date, home_team, away_team = df.loc[idx, ["Date", "HomeTeam", "AwayTeam"]]

row_filter = df.Date < date
row_filter = row_filter & ((df.HomeTeam.isin([home_team, away_team])) |
                           df.AwayTeam.isin([home_team, away_team]))

print(date, home_team, away_team)
df = df[row_filter]

1996-04-20 00:00:00 Torquay Chester


In [15]:
team = home_team
n_games = 1

temp_df = df[(df.HomeTeam == team) | (df.AwayTeam == team)]
temp_df = temp_df.sort_values("Date", ascending=False).head(n_games)[["HomeTeam", "AwayTeam", "FTR"]]
print(temp_df)
if len(temp_df) < n_games:
    print("NaN")
else:
    wins = ((temp_df.HomeTeam == team) & (temp_df.FTR == "A")) | \
           ((temp_df.AwayTeam == team) & (temp_df.FTR == "H"))
    win_percent = wins.sum() / n_games
    print(win_percent.astype(int))

      HomeTeam AwayTeam FTR
34973     Bury  Torquay   H
1
